In [114]:
import pandas as pd

In [115]:
gleif_data = pd.read_csv("../output/lei_info.txt", sep="|")
gleif_data.head()

c:\program files (x86)\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,hq_city,hq_country,hq_street1,hq_street2,id_lei,legal_city,legal_country,legal_name,legal_post,legal_region,legal_street1,legal_street2
0,NG-LA,NG,27 GERRARD ROAD,IKOYI,029200013A5N6ZD0F605,LAGOS,NG,AFRINVEST SECURITIES LIMITED,23401,NG-LA,27 GERRARD ROAD,IKOYI
1,NG-LA,NG,"9TH FLOOR BOOKSHOP HOUSE, 50/52 BROAD STREET",P.O. BOX 702 LAGOS,029200137F2K8AH5C573,LAGOS,NG,CAPITAL ASSETS LIMITED,23401,NG-LA,"9TH FLOOR BOOKSHOP HOUSE, 50/52 BROAD STREET",P.O. BOX 702 LAGOS
2,NG-LA,NG,"3RD FLOOR, UNTL BUILDING, 1, DAVIES STREET","OFF MARINA, LAGOS. P.O.BOX 1362, MARINA",029200038B4L4ZI1E579,LAGOS,NG,CAPITAL BANCORP PLC,23401,NG-LA,"3RD FLOOR, UNTL BUILDING, 1, DAVIES STREET","OFF MARINA, LAGOS. P.O.BOX 1362, MARINA"
3,NG-LA,NG,"PLOT 1626 C-E, SAKA JOJO STREET","OFF IDEJO STREET, VICTORIA ISLAND",029200048F0K9UI5A508,LAGOS,NG,CAPITAL EXPRESS SECURITIES LTD,23401,NG-LA,"PLOT 1626 C-E, SAKA JOJO STREET","OFF IDEJO STREET, VICTORIA ISLAND"
4,NG-LA,NG,"5, OKOTIE-EBOH STREET",IKOYI,029200222D9S2BC3J917,LAGOS,NG,CARDINALSTONE SECURITIES LIMITED,23401,NG-LA,"5, OKOTIE-EBOH STREET",IKOYI


In [116]:
panel_df = pd.read_csv("../output/test_run.txt", sep="|")
panel_df.match_tup.fillna("-1", inplace=True)
#panel_df.match_tup = panel_df.match_tup.apply(lambda x: x.replace(", N.W.'",""))
#panel_df.match_tup = panel_df.match_tup.apply(lambda x: x.replace(", SUITE 705'", ""))
#strip commas
#split by spaces
#select last two items in list
def split_tup(match_tup):
    if match_tup == "-1":
        return ["-1", "-1", "-1"]
    else:
        match_tup = match_tup.strip(",").split(" ")
        ret_list = [x.strip().strip(")") for x in match_tup]
    return ret_list
panel_df['match_score'] = panel_df.match_tup.apply(lambda x: split_tup(x)[-2].strip(","))
panel_df['match_index'] = panel_df.match_tup.apply(lambda x: split_tup(x)[-1])

#panel_df.match_index.unique()

In [117]:
#match panel records to gleif file to populate GLEIF lei
panel_df['lei_gleif'] = ""
panel_df['name_gleif'] = ""
panel_df['city_gleif'] = ""
panel_df['street1_gleif'] = ""
for index, row in panel_df.iterrows():
    if panel_df.at[index, 'match_index'] == "-1":
        panel_df.at[index, "lei_gleif"] = "-1"
    else:
        gleif_idx = int(panel_df.at[index, 'match_index'])
        gleif_lei = gleif_data.at[gleif_idx, 'id_lei']
        panel_df.at[index, 'lei_gleif'] = gleif_lei
        panel_df.at[index, 'name_gleif'] = gleif_data.at[gleif_idx, 'legal_name']
        panel_df.at[index, 'city_gleif'] = gleif_data.at[gleif_idx, 'legal_city']
        panel_df.at[index, 'street1_gleif'] = gleif_data.at[gleif_idx, 'legal_street1']
panel_df['lei_match'] = panel_df.apply(lambda x: "1" if x.lei_gleif == x.id_lei else "0", axis=1)

In [118]:
print(len(panel_df[(panel_df.id_lei==panel_df.lei_gleif)&(panel_df.id_lei!="-1")]))
print(len(panel_df[(panel_df.id_lei!=panel_df.lei_gleif)&(panel_df.id_lei!="-1")]))
panel_df[(panel_df.id_lei!=panel_df.lei_gleif)&(panel_df.id_lei!="-1")]

575
100


,agency_code,respondent_id,id_rssd,tax_id_dash,tax_id,arid,institution_id,id_lei,id_tax,nm_lgl,...,dt_start,match_string,match_tup,match_score,match_index,lei_gleif,name_gleif,city_gleif,street1_gleif,lei_match
63,1,24080,2907019,48-1238548,481238548,124080,2907019,549300KR4ZW8KJC82047,481238548,COMMUNITY FIRST NATIONAL BANK,...,20170804,COMMUNITY FIRST NATIONAL BANK MANHATTAN KS 215...,('Community First National Bank Manhattan KS 2...,95,1125205,549300KR4ZW8KJC82O47,Community First National Bank,Manhattan,215 South Seth Child Road,0
77,3,31469,245276,66-0259436,660259436,331469,245276,549300ZLPU75MU7O0T38,660259436,ORIENTAL BANK,...,20151231,ORIENTAL BANK SAN JUAN PR 254 MUNOZ RIVERA AVE...,"('CARLYLE EUROPE PARTNERS IV, L.P. WASHINGTON ...",86,20610,2138003N1DJLKVZ3B190,"CARLYLE EUROPE PARTNERS IV, L.P.",WASHINGTON D.C.,"THE CARLYLE GROUP 100 PENNSYLVANIA AVENUE, N.W.",0
115,9,197478,197478,95-2795851,952795851,9197478,197478,F28JOQ8OBWCFUYM0UX93,952795851,EAST WEST BANK,...,20151231,EAST WEST BANK PASADENA CA 135 NORTH LOS ROBLE...,"('HS EAST, LLP PITTSBURGH 2250 ROSWELL DRIVE'...",86,4725,21380024FHDAX4N84128,"HS EAST, LLP",PITTSBURGH,2250 ROSWELL DRIVE,0
127,3,264030355,4320108,26-4030355,264030355,3264030355,4320108,549300PGT5K5GACK2444,264030355,"POTOMAC MORTGAGE GROUP, INC.",...,20151231,"POTOMAC MORTGAGE GROUP, INC. FAIRFAX VA 0",('BENEFICIAL CONNECTICUT INC. WILMINGTON C/O ...,86,3142,213800694GO11UR95731,BENEFICIAL CONNECTICUT INC.,WILMINGTON,C/O THE CORPORATION TRUST COMPANY,0
171,3,58279,3433047,20-4877174,204877174,358279,3433047,549300SM6EJ5M7G8GP62,204877174,"COMMERCE BANK, THE",...,20180131,"COMMERCE BANK, THE EVANSVILLE IN 7312 EAGLE CR...",('BENEFICIAL CREDIT SERVICES OF MISSISSIPPI IN...,86,3145,213800EJQYMC7UWVV459,BENEFICIAL CREDIT SERVICES OF MISSISSIPPI INC.,WILMINGTON,C/O THE CORPORATION TRUST COMPANY,0
174,1,14688,2732,36-2245476,362245476,114688,2732,25490001F0A9F7BV6B05,362245476,"ALBANY BANK & TRUST CO., NATIONAL ASSOCIATION",...,20180315,"ALBANY BANK & TRUST CO., NATIONAL ASSOCIATION ...","('LION COMMERCIAL LLC WILMINGTON 108, WEST 13...",86,2253,21380087BAB79DFYRR32,LION COMMERCIAL LLC,WILMINGTON,"108, WEST 13TH STREET",0
193,3,12854,783648,35-0161790,350161790,312854,783648,549300SKTBTC2QFDKG02,350161790,CENTIER BANK,...,20161231,CENTIER BANK MERRILLVILLE IN 600 EAST 84TH AVENUE,('THE SAVINGS BANK LIFE INSURANCE COMPANY OF M...,86,7555,2138008UUIBDIGL25U15,THE SAVINGS BANK LIFE INSURANCE COMPANY OF MAS...,WOBURN,ONE LINSCOTT RD,0
219,3,22081,34434,64-0565094,640565094,322081,34434,5493007YWPQ2TU6YIG45,640565094,MAGNOLIA STATE BANK,...,20151231,MAGNOLIA STATE BANK BAY SPRINGS MS 28 HIGHWAY 528,('THE SAVINGS BANK LIFE INSURANCE COMPANY OF M...,86,7555,2138008UUIBDIGL25U15,THE SAVINGS BANK LIFE INSURANCE COMPANY OF MAS...,WOBURN,ONE LINSCOTT RD,0
241,3,412181683,3597930,41-2181683,412181683,3412181683,3597930,5493000F6NFDOVVZP043,412181683,"MORTGAGE SERVICES III, LLC",...,20161224,"MORTGAGE SERVICES III, LLC BLOOMINGTON IL 0",('BENEFICIAL CREDIT SERVICES INC. WILMINGTON ...,86,3143,213800NAN42LNYBD1Z52,BENEFICIAL CREDIT SERVICES INC.,WILMINGTON,C/O THE CORPORATION TRUST COMPANY,0
355,9,2735146,2735146,59-3531592,593531592,92735146,2735146,5Z1UQ1CWY0DQ3KJWDQ07,593531592,"TIAA, FSB",...,20170627,"TIAA, FSB JACKSONVILLE FL 501 RIVERSIDE AVE",('HOUSEHOLD FINANCIAL CENTER INC. BRANDON FL 6...,86,17004,213800PQ8HHWO88PM833,HOUSEHOLD FINANCIAL CENTER INC.,BRANDON,636 GRAND REGENCY BLVD,0


In [119]:
panel_df.columns

Index(['agency_code', 'respondent_id', 'id_rssd', 'tax_id_dash', 'tax_id',
       'arid', 'institution_id', 'id_lei', 'id_tax', 'nm_lgl', 'city',
       'state_cd', 'state_abbr_nm', 'state_home_cd', 'street_line1',
       'street_line2', 'cntry_nm', 'latest_row_ind', 'dt_end', 'dt_start',
       'match_string', 'match_tup', 'match_score', 'match_index', 'lei_gleif',
       'name_gleif', 'city_gleif', 'street1_gleif', 'lei_match'],
      dtype='object')

In [120]:
#reorder columns for name and address comparison
col_list = list(panel_df.columns.ravel())
col_list = ['agency_code',
 'respondent_id',
 'id_rssd',
 'tax_id_dash',
 'tax_id',
 'id_tax',
 'arid',
 'institution_id',
 'id_lei',
 'lei_gleif',
 'cntry_nm',
 'latest_row_ind',
 'dt_end',
 'dt_start',
 'nm_lgl',
 'name_gleif',
 'city',
 'city_gleif',
 'state_cd',
 'state_abbr_nm',
 'state_home_cd',
 'street_line1',
 'street1_gleif',
 'lei_match',
 'match_string',
 'match_tup',
 'match_score',
 'match_index',
 'lei_match']
panel_df = panel_df[col_list]

In [121]:
panel_df[panel_df.id_lei!="-1"].to_csv("../output/test_run_split.txt", sep="|")